<a href="https://colab.research.google.com/github/PyBeginner1/CarPricePredict/blob/main/UsedCarPricePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
import lightgbm as lgb

from sklearn.metrics import mean_squared_error


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
data = pd.read_csv('/content/drive/MyDrive/vehicles.csv')

In [5]:
data

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,7301591192,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,23590,2019.0,nissan,maxima s sedan 4d,good,6 cylinders,gas,32226.0,clean,other,1N4AA6AV6KC367801,fwd,NaN,sedan,NaN,https://images.craigslist.org/00o0o_iiraFnHg8q...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:31-0600
426876,7301591187,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,30590,2020.0,volvo,s60 t5 momentum sedan 4d,good,NaN,gas,12029.0,clean,other,7JR102FKXLG042696,fwd,NaN,sedan,red,https://images.craigslist.org/00x0x_15sbgnxCIS...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:29-0600
426877,7301591147,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,34990,2020.0,cadillac,xt4 sport suv 4d,good,NaN,diesel,4174.0,clean,other,1GYFZFR46LF088296,NaN,NaN,hatchback,white,https://images.craigslist.org/00L0L_farM7bxnxR...,Carvana is the safer way to buy a car During t...,NaN,wy,33.779214,-84.411811,2021-04-04T03:21:17-0600
426878,7301591140,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,28990,2018.0,lexus,es 350 sedan 4d,good,6 cylinders,gas,30112.0,clean,other,58ABK1GG4JU103853,fwd,NaN,sedan,silver,https://images.craigslist.org/00z0z_bKnIVGLkDT...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:11-0600


In [6]:
pd.set_option('display.max_columns', None)

In [7]:
data


,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,7301591192,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,23590,2019.0,nissan,maxima s sedan 4d,good,6 cylinders,gas,32226.0,clean,other,1N4AA6AV6KC367801,fwd,NaN,sedan,NaN,https://images.craigslist.org/00o0o_iiraFnHg8q...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:31-0600
426876,7301591187,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,30590,2020.0,volvo,s60 t5 momentum sedan 4d,good,NaN,gas,12029.0,clean,other,7JR102FKXLG042696,fwd,NaN,sedan,red,https://images.craigslist.org/00x0x_15sbgnxCIS...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:29-0600
426877,7301591147,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,34990,2020.0,cadillac,xt4 sport suv 4d,good,NaN,diesel,4174.0,clean,other,1GYFZFR46LF088296,NaN,NaN,hatchback,white,https://images.craigslist.org/00L0L_farM7bxnxR...,Carvana is the safer way to buy a car During t...,NaN,wy,33.779214,-84.411811,2021-04-04T03:21:17-0600
426878,7301591140,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,28990,2018.0,lexus,es 350 sedan 4d,good,6 cylinders,gas,30112.0,clean,other,58ABK1GG4JU103853,fwd,NaN,sedan,silver,https://images.craigslist.org/00z0z_bKnIVGLkDT...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:11-0600


Check for null/missing values

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 26 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   url           426880 non-null  object 
 2   region        426880 non-null  object 
 3   region_url    426880 non-null  object 
 4   price         426880 non-null  int64  
 5   year          425675 non-null  float64
 6   manufacturer  409234 non-null  object 
 7   model         421603 non-null  object 
 8   condition     252776 non-null  object 
 9   cylinders     249202 non-null  object 
 10  fuel          423867 non-null  object 
 11  odometer      422480 non-null  float64
 12  title_status  418638 non-null  object 
 13  transmission  424324 non-null  object 
 14  VIN           265838 non-null  object 
 15  drive         296313 non-null  object 
 16  size          120519 non-null  object 
 17  type          334022 non-null  object 
 18  pain

In [9]:
data.isnull().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year              1205
manufacturer     17646
model             5277
condition       174104
cylinders       177678
fuel              3013
odometer          4400
title_status      8242
transmission      2556
VIN             161042
drive           130567
size            306361
type             92858
paint_color     130203
image_url           68
description         70
county          426880
state                0
lat               6549
long              6549
posting_date        68
dtype: int64

In [10]:
data.isna().mean()

id              0.000000
url             0.000000
region          0.000000
region_url      0.000000
price           0.000000
year            0.002823
manufacturer    0.041337
model           0.012362
condition       0.407852
cylinders       0.416225
fuel            0.007058
odometer        0.010307
title_status    0.019308
transmission    0.005988
VIN             0.377254
drive           0.305863
size            0.717675
type            0.217527
paint_color     0.305011
image_url       0.000159
description     0.000164
county          1.000000
state           0.000000
lat             0.015342
long            0.015342
posting_date    0.000159
dtype: float64

Columns with 25% null values

In [11]:
data.isna().mean() > 0.20

id              False
url             False
region          False
region_url      False
price           False
year            False
manufacturer    False
model           False
condition        True
cylinders        True
fuel            False
odometer        False
title_status    False
transmission    False
VIN              True
drive            True
size             True
type             True
paint_color      True
image_url       False
description     False
county           True
state           False
lat             False
long            False
posting_date    False
dtype: bool

In [12]:
data.columns[data.isna().mean() > 0.20]

Index(['condition', 'cylinders', 'VIN', 'drive', 'size', 'type', 'paint_color',
       'county'],
      dtype='object')

Deleting all columns with over 25% null values

In [13]:
null_values = data.columns[data.isna().mean() > 0.20]

data = data.drop(null_values, axis =1)

In [14]:
data

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,7301591192,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,23590,2019.0,nissan,maxima s sedan 4d,gas,32226.0,clean,other,https://images.craigslist.org/00o0o_iiraFnHg8q...,Carvana is the safer way to buy a car During t...,wy,33.786500,-84.445400,2021-04-04T03:21:31-0600
426876,7301591187,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,30590,2020.0,volvo,s60 t5 momentum sedan 4d,gas,12029.0,clean,other,https://images.craigslist.org/00x0x_15sbgnxCIS...,Carvana is the safer way to buy a car During t...,wy,33.786500,-84.445400,2021-04-04T03:21:29-0600
426877,7301591147,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,34990,2020.0,cadillac,xt4 sport suv 4d,diesel,4174.0,clean,other,https://images.craigslist.org/00L0L_farM7bxnxR...,Carvana is the safer way to buy a car During t...,wy,33.779214,-84.411811,2021-04-04T03:21:17-0600
426878,7301591140,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,28990,2018.0,lexus,es 350 sedan 4d,gas,30112.0,clean,other,https://images.craigslist.org/00z0z_bKnIVGLkDT...,Carvana is the safer way to buy a car During t...,wy,33.786500,-84.445400,2021-04-04T03:21:11-0600


In [15]:
data.isnull().sum()

id                  0
url                 0
region              0
region_url          0
price               0
year             1205
manufacturer    17646
model            5277
fuel             3013
odometer         4400
title_status     8242
transmission     2556
image_url          68
description        70
state               0
lat              6549
long             6549
posting_date       68
dtype: int64

In [16]:
unneeded_columns = ['id','url','region_url','image_url', 'description','odometer','posting_date', 'manufacturer' ]

data = data.drop(unneeded_columns, axis =1)

In [17]:
data

,region,price,year,model,fuel,title_status,transmission,state,lat,long
0,prescott,6000,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN
1,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN
2,florida keys,21000,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN
3,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN
4,greensboro,4900,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
426875,wyoming,23590,2019.0,maxima s sedan 4d,gas,clean,other,wy,33.786500,-84.445400
426876,wyoming,30590,2020.0,s60 t5 momentum sedan 4d,gas,clean,other,wy,33.786500,-84.445400
426877,wyoming,34990,2020.0,xt4 sport suv 4d,diesel,clean,other,wy,33.779214,-84.411811
426878,wyoming,28990,2018.0,es 350 sedan 4d,gas,clean,other,wy,33.786500,-84.445400


Unique values in categorical columns

In [18]:
{column : len(data[column].unique()) for column in data.columns if data.dtypes[column] =='object'}

{'fuel': 6,
 'model': 29668,
 'region': 404,
 'state': 51,
 'title_status': 7,
 'transmission': 4}

In [19]:
data = data.drop('model', axis =1)

In [20]:
data


,region,price,year,fuel,title_status,transmission,state,lat,long
0,prescott,6000,NaN,NaN,NaN,NaN,az,NaN,NaN
1,fayetteville,11900,NaN,NaN,NaN,NaN,ar,NaN,NaN
2,florida keys,21000,NaN,NaN,NaN,NaN,fl,NaN,NaN
3,worcester / central MA,1500,NaN,NaN,NaN,NaN,ma,NaN,NaN
4,greensboro,4900,NaN,NaN,NaN,NaN,nc,NaN,NaN
...,...,...,...,...,...,...,...,...,...
426875,wyoming,23590,2019.0,gas,clean,other,wy,33.786500,-84.445400
426876,wyoming,30590,2020.0,gas,clean,other,wy,33.786500,-84.445400
426877,wyoming,34990,2020.0,diesel,clean,other,wy,33.779214,-84.411811
426878,wyoming,28990,2018.0,gas,clean,other,wy,33.786500,-84.445400


One hot encoding

In [21]:
def onehot_encode(df, columns, prefixes):
  df = df.copy()
  for column, prefix in zip(columns,prefixes):
    dummies = pd.get_dummies(df[column], prefix = prefix)
    df = pd.concat([df, dummies], axis = 1)
    df = df.drop(column, axis =1)
  return df

In [22]:
data = onehot_encode(data, ['fuel', 'region', 'state', 'title_status', 'transmission'], ['fuel', 'reg', 'state', 'title', 'transmission'])

In [23]:
data

,price,year,lat,long,fuel_diesel,fuel_electric,fuel_gas,fuel_hybrid,fuel_other,reg_SF bay area,reg_abilene,reg_akron / canton,reg_albany,reg_albuquerque,reg_altoona-johnstown,reg_amarillo,reg_ames,reg_anchorage / mat-su,reg_ann arbor,reg_annapolis,reg_appleton-oshkosh-FDL,reg_asheville,reg_ashtabula,reg_athens,reg_atlanta,reg_auburn,reg_augusta,reg_austin,reg_bakersfield,reg_baltimore,reg_baton rouge,reg_battle creek,reg_beaumont / port arthur,reg_bellingham,reg_bemidji,reg_bend,reg_billings,reg_binghamton,reg_birmingham,reg_bismarck,reg_bloomington,reg_bloomington-normal,reg_boise,reg_boone,reg_boston,reg_boulder,reg_bowling green,reg_bozeman,reg_brainerd,reg_brownsville,reg_brunswick,reg_buffalo,reg_butte,reg_cape cod / islands,reg_catskills,reg_cedar rapids,reg_central NJ,reg_central louisiana,reg_central michigan,reg_champaign urbana,reg_charleston,reg_charlotte,reg_charlottesville,reg_chattanooga,reg_chautauqua,reg_chicago,reg_chico,reg_chillicothe,reg_cincinnati,reg_clarksville,reg_cleveland,reg_clovis / portales,reg_college station,reg_colorado springs,reg_columbia,reg_columbia / jeff city,reg_columbus,reg_cookeville,reg_corpus christi,reg_corvallis/albany,reg_cumberland valley,reg_dallas / fort worth,reg_danville,reg_dayton / springfield,reg_daytona beach,reg_decatur,reg_deep east texas,reg_del rio / eagle pass,reg_delaware,reg_denver,reg_des moines,reg_detroit metro,reg_dothan,reg_dubuque,reg_duluth / superior,reg_east idaho,reg_east oregon,reg_eastern CO,reg_eastern CT,reg_eastern NC,reg_eastern kentucky,reg_eastern montana,reg_eastern panhandle,reg_eastern shore,reg_eau claire,reg_el paso,reg_elko,reg_elmira-corning,reg_erie,reg_eugene,reg_evansville,reg_fairbanks,reg_fargo / moorhead,reg_farmington,reg_fayetteville,reg_finger lakes,reg_flagstaff / sedona,reg_flint,reg_florence,reg_florence / muscle shoals,reg_florida keys,reg_fort collins / north CO,reg_fort dodge,reg_fort smith,"reg_fort smith, AR",reg_fort wayne,reg_frederick,reg_fredericksburg,reg_fresno / madera,reg_ft myers / SW florida,reg_gadsden-anniston,reg_gainesville,reg_galveston,reg_glens falls,reg_gold country,reg_grand forks,reg_grand island,reg_grand rapids,reg_great falls,reg_green bay,reg_greensboro,reg_greenville / upstate,reg_gulfport / biloxi,reg_hanford-corcoran,reg_harrisburg,reg_harrisonburg,reg_hartford,reg_hattiesburg,reg_hawaii,reg_heartland florida,reg_helena,reg_hickory / lenoir,reg_high rockies,reg_hilton head,reg_holland,reg_houma,reg_houston,reg_hudson valley,reg_humboldt county,reg_huntington-ashland,reg_huntsville / decatur,reg_imperial county,reg_indianapolis,reg_inland empire,reg_iowa city,reg_ithaca,reg_jackson,reg_jacksonville,reg_janesville,reg_jersey shore,reg_jonesboro,reg_joplin,reg_kalamazoo,reg_kalispell,reg_kansas city,"reg_kansas city, MO",reg_kenai peninsula,reg_kennewick-pasco-richland,reg_kenosha-racine,reg_killeen / temple / ft hood,reg_kirksville,reg_klamath falls,reg_knoxville,reg_kokomo,reg_la crosse,reg_la salle co,reg_lafayette,reg_lafayette / west lafayette,reg_lake charles,reg_lake of the ozarks,reg_lakeland,reg_lancaster,reg_lansing,reg_laredo,reg_las cruces,reg_las vegas,reg_lawrence,reg_lawton,reg_lehigh valley,reg_lewiston / clarkston,reg_lexington,reg_lima / findlay,reg_lincoln,reg_little rock,reg_logan,reg_long island,reg_los angeles,reg_louisville,reg_lubbock,reg_lynchburg,reg_macon / warner robins,reg_madison,reg_maine,reg_manhattan,reg_mankato,reg_mansfield,reg_mason city,reg_mattoon-charleston,reg_mcallen / edinburg,reg_meadville,reg_medford-ashland,reg_memphis,reg_mendocino county,reg_merced,reg_meridian,reg_milwaukee,reg_minneapolis / st paul,reg_missoula,reg_mobile,reg_modesto,reg_mohave county,reg_monroe,reg_monterey bay,reg_montgomery,reg_morgantown,reg_moses lake,reg_muncie / anderson,reg_muskegon,reg_myrtle beach,reg_nashville,reg_new hampshire,reg_new haven,reg_new orleans,reg_new river valley,reg_new york city,reg_norfolk / hampton roads,reg_north central FL,reg_north dakota,reg_north jerse

In [24]:
for column in data.columns:
  data[column] = data[column].fillna(data[column].mean())

In [25]:
data.isna().sum()

price                     0
year                      0
lat                       0
long                      0
fuel_diesel               0
                         ..
title_rebuilt             0
title_salvage             0
transmission_automatic    0
transmission_manual       0
transmission_other        0
Length: 473, dtype: int64

In [26]:
data.isna().sum().sum()

0

Splitting

In [27]:
X = data.drop('price', axis =1)
Y = data['price']

In [28]:
X

,year,lat,long,fuel_diesel,fuel_electric,fuel_gas,fuel_hybrid,fuel_other,reg_SF bay area,reg_abilene,reg_akron / canton,reg_albany,reg_albuquerque,reg_altoona-johnstown,reg_amarillo,reg_ames,reg_anchorage / mat-su,reg_ann arbor,reg_annapolis,reg_appleton-oshkosh-FDL,reg_asheville,reg_ashtabula,reg_athens,reg_atlanta,reg_auburn,reg_augusta,reg_austin,reg_bakersfield,reg_baltimore,reg_baton rouge,reg_battle creek,reg_beaumont / port arthur,reg_bellingham,reg_bemidji,reg_bend,reg_billings,reg_binghamton,reg_birmingham,reg_bismarck,reg_bloomington,reg_bloomington-normal,reg_boise,reg_boone,reg_boston,reg_boulder,reg_bowling green,reg_bozeman,reg_brainerd,reg_brownsville,reg_brunswick,reg_buffalo,reg_butte,reg_cape cod / islands,reg_catskills,reg_cedar rapids,reg_central NJ,reg_central louisiana,reg_central michigan,reg_champaign urbana,reg_charleston,reg_charlotte,reg_charlottesville,reg_chattanooga,reg_chautauqua,reg_chicago,reg_chico,reg_chillicothe,reg_cincinnati,reg_clarksville,reg_cleveland,reg_clovis / portales,reg_college station,reg_colorado springs,reg_columbia,reg_columbia / jeff city,reg_columbus,reg_cookeville,reg_corpus christi,reg_corvallis/albany,reg_cumberland valley,reg_dallas / fort worth,reg_danville,reg_dayton / springfield,reg_daytona beach,reg_decatur,reg_deep east texas,reg_del rio / eagle pass,reg_delaware,reg_denver,reg_des moines,reg_detroit metro,reg_dothan,reg_dubuque,reg_duluth / superior,reg_east idaho,reg_east oregon,reg_eastern CO,reg_eastern CT,reg_eastern NC,reg_eastern kentucky,reg_eastern montana,reg_eastern panhandle,reg_eastern shore,reg_eau claire,reg_el paso,reg_elko,reg_elmira-corning,reg_erie,reg_eugene,reg_evansville,reg_fairbanks,reg_fargo / moorhead,reg_farmington,reg_fayetteville,reg_finger lakes,reg_flagstaff / sedona,reg_flint,reg_florence,reg_florence / muscle shoals,reg_florida keys,reg_fort collins / north CO,reg_fort dodge,reg_fort smith,"reg_fort smith, AR",reg_fort wayne,reg_frederick,reg_fredericksburg,reg_fresno / madera,reg_ft myers / SW florida,reg_gadsden-anniston,reg_gainesville,reg_galveston,reg_glens falls,reg_gold country,reg_grand forks,reg_grand island,reg_grand rapids,reg_great falls,reg_green bay,reg_greensboro,reg_greenville / upstate,reg_gulfport / biloxi,reg_hanford-corcoran,reg_harrisburg,reg_harrisonburg,reg_hartford,reg_hattiesburg,reg_hawaii,reg_heartland florida,reg_helena,reg_hickory / lenoir,reg_high rockies,reg_hilton head,reg_holland,reg_houma,reg_houston,reg_hudson valley,reg_humboldt county,reg_huntington-ashland,reg_huntsville / decatur,reg_imperial county,reg_indianapolis,reg_inland empire,reg_iowa city,reg_ithaca,reg_jackson,reg_jacksonville,reg_janesville,reg_jersey shore,reg_jonesboro,reg_joplin,reg_kalamazoo,reg_kalispell,reg_kansas city,"reg_kansas city, MO",reg_kenai peninsula,reg_kennewick-pasco-richland,reg_kenosha-racine,reg_killeen / temple / ft hood,reg_kirksville,reg_klamath falls,reg_knoxville,reg_kokomo,reg_la crosse,reg_la salle co,reg_lafayette,reg_lafayette / west lafayette,reg_lake charles,reg_lake of the ozarks,reg_lakeland,reg_lancaster,reg_lansing,reg_laredo,reg_las cruces,reg_las vegas,reg_lawrence,reg_lawton,reg_lehigh valley,reg_lewiston / clarkston,reg_lexington,reg_lima / findlay,reg_lincoln,reg_little rock,reg_logan,reg_long island,reg_los angeles,reg_louisville,reg_lubbock,reg_lynchburg,reg_macon / warner robins,reg_madison,reg_maine,reg_manhattan,reg_mankato,reg_mansfield,reg_mason city,reg_mattoon-charleston,reg_mcallen / edinburg,reg_meadville,reg_medford-ashland,reg_memphis,reg_mendocino county,reg_merced,reg_meridian,reg_milwaukee,reg_minneapolis / st paul,reg_missoula,reg_mobile,reg_modesto,reg_mohave county,reg_monroe,reg_monterey bay,reg_montgomery,reg_morgantown,reg_moses lake,reg_muncie / anderson,reg_muskegon,reg_myrtle beach,reg_nashville,reg_new hampshire,reg_new haven,reg_new orleans,reg_new river valley,reg_new york city,reg_norfolk / hampton roads,reg_north central FL,reg_north dakota,reg_north jersey,reg_

In [29]:
Y

0          6000
1         11900
2         21000
3          1500
4          4900
          ...  
426875    23590
426876    30590
426877    34990
426878    28990
426879    30590
Name: price, Length: 426880, dtype: int64

Scaling

In [30]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [31]:
X

array([[-2.40893630e-14,  5.35676359e-13,  2.26060278e-12, ...,
        -1.92987652e+00, -2.50039199e-01, -4.14861033e-01],
       [-2.40893630e-14,  5.35676359e-13,  2.26060278e-12, ...,
        -1.92987652e+00, -2.50039199e-01, -4.14861033e-01],
       [-2.40893630e-14,  5.35676359e-13,  2.26060278e-12, ...,
        -1.92987652e+00, -2.50039199e-01, -4.14861033e-01],
       ...,
       [ 9.28597655e-01, -8.13368423e-01,  5.67206825e-01, ...,
        -1.92987652e+00, -2.50039199e-01,  2.41044572e+00],
       [ 7.16705377e-01, -8.12111467e-01,  5.65363708e-01, ...,
        -1.92987652e+00, -2.50039199e-01,  2.41044572e+00],
       [ 8.22651516e-01, -8.13368423e-01,  5.67206825e-01, ...,
        -1.92987652e+00, -2.50039199e-01,  2.41044572e+00]])

Train & Test split

In [53]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 2)

In [54]:
X.shape, X_train.shape, X_test.shape

((426880, 472), (298816, 472), (128064, 472))

Training the model ---> Linear Regression & light Gradient Boosting Machine

In [55]:
model = LinearRegression()
model.fit(X_train, Y_train)

linear_predict = model.predict(X_test)


In [56]:
model_x = lgb.LGBMRegressor(boosting_type = 'gbdt', num_leaves = 31, n_estimators = 100, reg_lambda = 1.0)
model_x.fit(X_train, Y_train)

gradient_predict = model_x.predict(X_test)



In [57]:
score_lin = np.sqrt(mean_squared_error(Y_test, linear_predict))
score_xgb = np.sqrt(mean_squared_error(Y_test, gradient_predict))

In [58]:
print('Linear Regression RMSE: ',score_lin)
print('Gradient Boosting RMSE: ' ,score_xgb)

Linear Regression RMSE:  5.9492946762291384e+16
Gradient Boosting RMSE:  14440965.80217639
